Импортируем библиотеки

In [1]:
import numpy as np
import time
import random
!pip install pycuda
import pycuda.autoinit
import pycuda.gpuarray as gpuarray
from pycuda.curandom import rand as curandom
from pycuda import driver as dr
from pycuda.compiler import SourceModule

     |████████████████████████████████| 1.7 MB 4.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 75 kB 4.6 MB/s 
     |████████████████████████████████| 67 kB 5.7 MB/s 
  Created wheel for pycuda: filename=pycuda-2021.1-cp37-cp37m-linux_x86_64.whl size=626634 sha256=d7d30fc08acdbd243c77fa6fdeeae3b1d674a6b2c3f32580c1c618e062799edf
  Stored in directory: /root/.cache/pip/wheels/c4/ef/49/dc6a5feb8d980b37c83d465ecab24949a6aa19458522a9e001
  Created wheel for pytools: filename=pytools-2022.1-py2.py3-none-any.whl size=63426 sha256=32d6ff465afaa0ee93cf585bb9fdb5a3131fb81011ac7c42074e657dcd4b0c77
  Stored in directory: /root/.cache/pip/wheels/b4/11/0e/75dfe0884733df6a47170da81d3f1b28276ff7bc4315287190
Successfully built pycuda pytools


Зададим количество элементов после запятой

In [18]:
N = 10000

Зададим функцию для выполнения на ядре

In [19]:
kernel = SourceModule(
    """
    __global__ void foundpi(double *x, double *y, int *points_in_circle, const int N){
        int j = blockIdx.x * blockDim.x + threadIdx.x;
        int c = 0;
        for (int i = j; i < N; i += gridDim.x * blockDim.x) {
            if (x[i]*x[i] + y[i]*y[i] <= 1) {
            c+=1;
            }
        }
        atomicAdd(points_in_circle, c);
    }
    """)

Зададим функцию, которая будет проводить рассчёты числа Pi методом Монте-Карло на CPU и GPU, а также выводить все необходимые метрики

<!-- Используем N - количество чисел после запятой, чтобы определить количество раз, сколько необходимо исполнить цикл для вычисления числа pi. В этой же функции пропишем реализацию вычислений на CPU и GPU соответственно -->

In [24]:
def main():    
  n_vals = []
  cpu_ts = []
  gpu_ts = []
  acc = []
  data = [cpu_ts, gpu_ts, n_vals, acc]
  for power in range(14, 20):  # степени 2
    N = 2 ** power  # power, it's a power! OnePuMa
    print(N)
    start_time = time.time()
    points_in_circle = 0
    x = np.zeros((N, 1), dtype=np.float64)
    y = np.zeros((N, 1), dtype=np.float64)
    for i in range(N):
      x[i] = random.uniform(-1, 1)
      y[i] = random.uniform(-1, 1)

    for i in range(N):
      if x[i] ** 2 + y[i] ** 2 <= 1:
        points_in_circle = points_in_circle + 1

    pi = 4 * points_in_circle / N
    end_time = time.time()
    cpu_t = end_time - start_time
    err = np.abs(np.pi - pi)

    start_time = time.time()
    gpu_points_in_circle = gpuarray.zeros((1,), dtype=np.int32)
    gpu_points_in_circle = gpu_points_in_circle.get()

    gpu_x = curandom((N,), dtype=np.double).get().astype(np.double)
    gpu_y = curandom((N,), dtype=np.double).get().astype(np.double)
    pi_calc = kernel.get_function("foundpi")
    pi_calc(dr.In(gpu_x), dr.In(gpu_y), dr.Out(gpu_points_in_circle), np.int32(N), block=(128, 1, 1),
            grid=(int(N / (128 ** 2)), 1))
    dr.Context.synchronize()

    gpu_pi = 4 * gpu_points_in_circle[0] / N
    end_time = time.time()
    gpu_t = end_time - start_time
    gpu_err = np.abs(np.pi - gpu_pi)

    acceleration = cpu_t / gpu_t

    print(pi)
    print('Cpu time: ', cpu_t, '. Error: ', err)

    print(gpu_pi)
    print('Gpu time: ', gpu_t, '. Error: ', gpu_err)
    print('T(cpu)/T(gpu):', acceleration,"\n")

    cpu_ts.append(cpu_t)
    gpu_ts.append(gpu_t)
    n_vals.append(N)
    acc.append(acceleration)
  print(data)

Запустим заданную функцию

In [25]:
if __name__ == '__main__':
  main()

16384
3.16357421875
Cpu time:  0.12070608139038086 . Error:  0.021981565160206884
3.155517578125
Gpu time:  0.20677804946899414 . Error:  0.013924924535206884
T(cpu)/T(gpu): 0.5837470742196958 

32768
3.1385498046875
Cpu time:  0.22400450706481934 . Error:  0.003042848902293116
3.12646484375
Gpu time:  0.2054450511932373 . Error:  0.015127809839793116
T(cpu)/T(gpu): 1.090337809374271 

65536
3.14678955078125
Cpu time:  0.433635950088501 . Error:  0.005196897191456884
3.15301513671875
Gpu time:  0.20337581634521484 . Error:  0.011422483128956884
T(cpu)/T(gpu): 2.1321903355138216 

131072
3.140350341796875
Cpu time:  0.7971701622009277 . Error:  0.001242311792918116
3.13427734375
Gpu time:  0.2006235122680664 . Error:  0.007315309839793116
T(cpu)/T(gpu): 3.973463295447523 

262144
3.1380615234375
Cpu time:  1.6409132480621338 . Error:  0.003531130152293116
3.142181396484375
Gpu time:  0.20459818840026855 . Error:  0.000588742894581884
T(cpu)/T(gpu): 8.020174865145481 

524288
3.144020080